<a href="https://colab.research.google.com/github/sangeeta05071997/python-machine-learning/blob/main/resume_parsing_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A MAJOR PROJECT REPORT ON
“Automated Resume Parsing Using NLP and Cosine Similarity”

SUBMITTED BY:
Batch 1515 CP-26 Team-A
(Sangeeta Suresh Singh)

UNDER THE GUIDANCE OF:
Aniket Manwatkar and Saksham Sir

SUBMITTED TO:
EVOASTRA VENTURES PVT LTD

In [263]:
# Install Required Packages
!pip install pdfminer.six python-docx spacy
!python -m spacy download en_core_web_sm
!pip install pdfminer.six python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [264]:
import os
import re
from pdfminer.high_level import extract_text as extract_text_from_pdf
from docx import Document
import spacy

Step 2: Extracting Key Information

In [265]:
# Step 1:
from google.colab import files
import os

uploaded = files.upload()
resume_files = list(uploaded.keys())  # list of all uploaded resume file names
print("Uploaded files:", resume_files)

# Step 2: Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Extract Text from DOCX
def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    return '\n'.join([para.text for para in doc.paragraphs])

# Unified Text Extractor
def extract_resume_text(file_path):
    ext = os.path.splitext(file_path)[-1].lower()
    if ext == ".pdf":
        return extract_text_from_pdf(file_path)
    elif ext == ".docx":
        return extract_text_from_docx(file_path)
    else:
        raise ValueError("Unsupported file type. Only PDF and DOCX are supported.")

Saving Priya_Sharma_Data_Analyst_Resume.docx to Priya_Sharma_Data_Analyst_Resume (3).docx
Saving Saritha_SV.pdf to Saritha_SV (5).pdf
Saving NikhilYCV1.pdf to NikhilYCV1 (5).pdf
Saving ASNA Resume.pdf to ASNA Resume (5).pdf
Saving koushikresume.pdf to koushikresume (5).pdf
Saving Avanish_Kumar.pdf to Avanish_Kumar (5).pdf
Saving 23N0064-1.pdf to 23N0064-1 (5).pdf
Saving Resume 1.pdf to Resume 1 (6).pdf
Uploaded files: ['Priya_Sharma_Data_Analyst_Resume (3).docx', 'Saritha_SV (5).pdf', 'NikhilYCV1 (5).pdf', 'ASNA Resume (5).pdf', 'koushikresume (5).pdf', 'Avanish_Kumar (5).pdf', '23N0064-1 (5).pdf', 'Resume 1 (6).pdf']


In [266]:
# Step 3: Extract Text from Resume (PDF or words)
from pdfminer.high_level import extract_text as extract_pdf
from docx import Document

def extract_text(file_path):
    if file_path.endswith(".pdf"):
        return extract_pdf(file_path)
    elif file_path.endswith(".docx"):
        doc = Document(file_path)
        return "\n".join([para.text for para in doc.paragraphs])
    else:
        return "unsupported file type"

In [267]:
# Step 4: Extract Name (Top of Resume or NER)
from spacy.matcher import Matcher

def extract_name(text):
    nlp = spacy.load('en_core_web_sm')
    match = Matcher(nlp.vocab)

    # Define name patterns
    patterns = [
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}],  # First name, Middle name, and Last name
        [{'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}, {'POS': 'PROPN'}]  # First name, Middle name, Middle name, and Last name
        # Add more patterns as needed
    ]

    for pattern in patterns:
        match.add('NAME', patterns=[pattern])

    doc = nlp(text)
    matches = match(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text
    return None

In [268]:
# Step 5: Extract Email
# import re for this part

def extract_email(text):
    match = re.search(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', text)
    return match.group() if match else None

In [269]:
# Step 6: Extract Phone Number
def extract_phone(text):
    match = re.search(r'(\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}', text)
    return match.group() if match else None

In [270]:
# Step 7: Extract Skills (Using List)
skills = ["Python", "Java", "Machine Learning","SQL","Power BI","Statistics","MS Excel","Management", "Data Analysis", "Project Management"]
def extract_skills(text, skill_list):
    found = []
    text = text.lower()
    for skill in skill_list:
        if skill.lower() in text:
            found.append(skill)
    return list(set(found))  # remove duplicates

In [271]:
# Step 8: Extract Education
def extract_education(text):
    edu_keywords = ['Bachelor', 'Master', 'B.Tech', 'M.Tech', 'B.Sc', 'M.Sc', 'Ph.D', 'B.E']
    degrees = []
    for keyword in edu_keywords:
        matches = re.findall(rf"{keyword}.*\d{{4}}", text)
        degrees.extend(matches)
    return degrees

In [272]:
# Step 9: Extract Experience
def extract_experience(text):
    experience = []
    exp_section = re.findall(r'(?:Experience|Employment|Work History).*?(?=Education|Skills|$)', text, re.IGNORECASE | re.DOTALL)
    if exp_section:
        jobs = re.findall(r'[A-Z][a-zA-Z\s]+.*?\d{4}[-–]\d{4}', exp_section[0])
        experience.extend(jobs)
    return experience

In [273]:
# Step 10: Wrap Everything in a Function
def parse_resume(file_path, skills_list):
    text = extract_resume_text(file_path)
    return {
        "Name": extract_name(text),
        "Email": extract_email(text),
        "Phone": extract_phone(text),
        "Skills": extract_skills(text, skills_list),
        "Education": extract_education(text),
        "Experience": extract_experience(text)
    }

In [274]:
for file in resume_files:
  resume_data = parse_resume(file, skills)
  print(resume_data)

{'Name': 'Priya Sharma', 'Email': 'priyasharma89@example.com', 'Phone': '+91-9876543210', 'Skills': ['Power BI', 'Statistics', 'Python', 'SQL'], 'Education': [], 'Experience': []}
{'Name': 'SARITHA VEGGALAM', 'Email': 'sarithaveggalam2004@gmail.com', 'Phone': '+91 9381199850', 'Skills': ['Java', 'Machine Learning', 'Python', 'SQL'], 'Education': [], 'Experience': []}
{'Name': 'Professional Summary', 'Email': 'snnikhil2003@gmail.com', 'Phone': '+91 9182131485', 'Skills': ['Machine Learning', 'Data Analysis', 'Java', 'SQL', 'Python', 'Management'], 'Education': [], 'Experience': []}
{'Name': 'SYEDA ASNA', 'Email': 'asnam7111@gmail.com', 'Phone': '9945071686', 'Skills': ['Machine Learning', 'Java', 'SQL', 'Python', 'Management'], 'Education': [], 'Experience': []}
{'Name': 'Thottempudi Nagasai', 'Email': 'tkkoushikinin@gmail.com', 'Phone': '+91 9182486078', 'Skills': ['Java', 'Machine Learning', 'Management', 'SQL'], 'Education': [], 'Experience': []}
{'Name': 'Avanish Kumar', 'Email': 'a

Step 3

In [275]:
!pip install spacy pdfminer.six python-docx
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Step 3: Matching with Job Description

In [276]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [277]:
# Sample Extracted Resume Skills (Simulating Step 2 Output)
resumes = [
    "Java, HTML, Python, Machine Learning, SQL",
    "Machine Learning, Data Analysis, Java, SQL, Python, Management",
    "Machine Learning, Java, SQL, Python, Management",
    "Java, Machine Learning, Management, SQL",
    "Management, Python, SQL",
    "Power BI, Machine Learning, Data Analysis, SQL, Statistics, MS Excel, Python",
    "Power BI, Machine Learning, SQL, Statistics, MS Excel, Python, Management"
    ]

# Data Analyst Job Description
job_description ="""
We are seeking a Data Analyst with expertise in Python, SQL, Excel, and data visualization tools like Tableau or Power BI.
The ideal candidate should be familiar with data cleaning, statistical analysis, and reporting dashboards.
Skills in Pandas, NumPy, or Scikit-learn are a plus.
"""

TF-IDF Vectorization + Cosine Similarity

Step 4: Ranking Candidates

In [278]:
parsed_resumes = []  # Store all parsed resume dictionaries here

for file in resume_files:
    resume_data = parse_resume(file, skills)
    parsed_resumes.append(resume_data)

resume_names = [r['Name'] for r in parsed_resumes]
resume_skills = [" ".join(r['Skills']) for r in parsed_resumes]

job_description = """
We are seeking a Data Analyst with expertise in Python, SQL, Excel, and data visualization tools like Tableau or Power BI.
The ideal candidate should be familiar with data cleaning, statistical analysis, and reporting dashboards.
Skills in Pandas, NumPy, or Scikit-learn are a plus.
"""

#  TF-IDF + Cosine Similarity
documents = [job_description] + resume_skills
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
similarity_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()

# Combine with "Resume X - Name" labels
resume_labels = [f"Resume {i+1} - {name}" for i, name in enumerate(resume_names)]

# Create DataFrame and sort
df = pd.DataFrame({
    "Resume": resume_labels,
    "Match %": (similarity_scores * 100).round(2)
}).sort_values(by="Match %", ascending=False).reset_index(drop=True)

print(df)

                            Resume  Match %
0           Resume 7 - Gagan Kumar    25.26
1  Resume 3 - Professional Summary    21.14
2          Resume 1 - Priya Sharma    12.51
3       Resume 8 - Sangeeta Suresh    12.20
4         Resume 6 - Avanish Kumar     5.61
5      Resume 2 - SARITHA VEGGALAM     4.02
6            Resume 4 - SYEDA ASNA     3.58
7   Resume 5 - Thottempudi Nagasai     1.71


In [280]:
# Step 6: Filter by threshold (Match % > 20)
filtered_df = df[df["Match %"] > 12].reset_index(drop=True)

# Show only top matching resumes
print(filtered_df)

                            Resume  Match %
0           Resume 7 - Gagan Kumar    25.26
1  Resume 3 - Professional Summary    21.14
2          Resume 1 - Priya Sharma    12.51
3       Resume 8 - Sangeeta Suresh    12.20
